# Assignment 2

Before working on this assignment please read these instructions fully. In the submission area, you will notice that you can click the link to **Preview the Grading** for each step of the assignment. This is the criteria that will be used for peer grading. Please familiarize yourself with the criteria before beginning the assignment.

An NOAA dataset has been stored in the file `data/C2A2_data/BinnedCsvs_d400/fb441e62df2d58994928907a91895ec62c2c42e6cd075c2700843b89.csv`. This is the dataset to use for this assignment. Note: The data for this assignment comes from a subset of The National Centers for Environmental Information (NCEI) [Daily Global Historical Climatology Network](https://www1.ncdc.noaa.gov/pub/data/ghcn/daily/readme.txt) (GHCN-Daily). The GHCN-Daily is comprised of daily climate records from thousands of land surface stations across the globe.

Each row in the assignment datafile corresponds to a single observation.

The following variables are provided to you:

* **id** : station identification code
* **date** : date in YYYY-MM-DD format (e.g. 2012-01-24 = January 24, 2012)
* **element** : indicator of element type
    * TMAX : Maximum temperature (tenths of degrees C)
    * TMIN : Minimum temperature (tenths of degrees C)
* **value** : data value for element (tenths of degrees C)

For this assignment, you must:

1. Read the documentation and familiarize yourself with the dataset, then write some python code which returns a line graph of the record high and record low temperatures by day of the year over the period 2005-2014. The area between the record high and record low temperatures for each day should be shaded.
2. Overlay a scatter of the 2015 data for any points (highs and lows) for which the ten year record (2005-2014) record high or record low was broken in 2015.
3. Watch out for leap days (i.e. February 29th), it is reasonable to remove these points from the dataset for the purpose of this visualization.
4. Make the visual nice! Leverage principles from the first module in this course when developing your solution. Consider issues such as legends, labels, and chart junk.

The data you have been given is near **Ann Arbor, Michigan, United States**, and the stations the data comes from are shown on the map below.

In [ ]:
import matplotlib.pyplot as plt
import mplleaflet
import pandas as pd
import numpy as np

def leaflet_plot_stations(binsize, hashid):

    df = pd.read_csv('data/C2A2_data/BinSize_d{}.csv'.format(binsize))

    station_locations_by_hash = df[df['hash'] == hashid]

    lons = station_locations_by_hash['LONGITUDE'].tolist()
    lats = station_locations_by_hash['LATITUDE'].tolist()

    plt.figure(figsize=(8,8))

    plt.scatter(lons, lats, c='r', alpha=0.7, s=200)

    return mplleaflet.display()

leaflet_plot_stations(400,'fb441e62df2d58994928907a91895ec62c2c42e6cd075c2700843b89')

temperatureDataRaw = pd.read_csv('data/C2A2_data/BinnedCsvs_d400/fb441e62df2d58994928907a91895ec62c2c42e6cd075c2700843b89.csv')

temperatureData = temperatureDataRaw.where(~temperatureDataRaw['Date'].str.contains('02-29')).dropna()
temperatureData['Data_Value'] = temperatureData['Data_Value'] / 10

temperatureData20052014 = temperatureData.where(~temperatureData['Date'].str.contains('2015')).dropna()
temperatureData20052014[['YY', 'MM/DD']] = temperatureData20052014.Date.str.split("-", n=1, expand=True)

temperatureData2015 = temperatureData.where(temperatureData['Date'].str.contains('2015')).dropna()
temperatureData2015[['YY', 'MM/DD']] = temperatureData2015.Date.str.split("-", n=1, expand=True)

maxTemperatureData20052014 = temperatureData20052014.groupby(['MM/DD']).max().drop(['ID', 'Date', 'Element', 'YY'], axis=1)
minTemperatureData20052014 = temperatureData20052014.groupby(['MM/DD']).min().drop(['ID', 'Date', 'Element', 'YY'], axis=1)

maxTemperatureData2015 = temperatureData2015.groupby(['MM/DD']).max().drop(['ID', 'Date', 'Element', 'YY'], axis=1)
minTemperatureData2015 = temperatureData2015.groupby(['MM/DD']).min().drop(['ID', 'Date', 'Element', 'YY'], axis=1)

ax = maxTemperatureData20052014.plot(x=maxTemperatureData20052014.index, y='Data_Value', label='Record high', c='r', alpha=0.5)
minTemperatureData20052014.plot(ax=ax, y='Data_Value', label='Record low', c='b', alpha=0.5)
plt.scatter(np.where(maxTemperatureData2015['Data_Value'] > maxTemperatureData20052014['Data_Value']),
            y=maxTemperatureData2015[(maxTemperatureData2015['Data_Value'] > maxTemperatureData20052014['Data_Value'])].values,
            s=25, label='2015 record high', c='r')
plt.scatter(np.where(minTemperatureData2015['Data_Value'] < minTemperatureData20052014['Data_Value']),
            y=minTemperatureData2015[(minTemperatureData2015['Data_Value'] < minTemperatureData20052014['Data_Value'])].values,
            s=25, label='2015 record low', c='b')

plt.fill_between(np.arange(0,365), maxTemperatureData20052014['Data_Value'], minTemperatureData20052014['Data_Value'], facecolor='k', alpha=0.25)
plt.legend(['Record high', 'Record low', '2015 record high', '2015 record low'])
plt.title('Record temperatures for 2005-2014 per day and 2015 extremums', size=14)
plt.ylabel('Temperature ($^\circ C$)', size=14)
plt.xticks(np.arange(0,365,31), ['Jan','Feb', 'Mar','Apr','May','Jun','Jul','Aug','Sep','Oct','Nov','Dec'])
plt.xlabel('Months', size=14)
plt.gcf().set_size_inches(14, 8)
plt.show()